In [1]:
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
from google.colab import userdata
import os
#OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY



In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages.ai import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from typing import Literal, Annotated
from collections.abc import Iterable
from random import randint
from langchain_core.messages.tool import ToolMessage
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict

In [4]:
class OrderState(TypedDict):
    """State representing the customer's order conversation."""
    messages: Annotated[list, add_messages]
    order: list[str]
    finished: bool

# System instruction
BARISTABOT_SYSINT = (
    "system",
    "You are a BaristaBot, an interactive cafe ordering system. A human will talk to you about the "
    "available products you have and you will answer any questions about menu items (and only about "
    "menu items - no off-topic discussion, but you can chat about the products and their history). "
    "The customer will place an order for 1 or more items from the menu, which you will structure "
    "and send to the ordering system after confirming the order with the human. "
    "\n\n"
    "Add items to the customer's order with add_to_order, and reset the order with clear_order. "
    "To see the contents of the order so far, call get_order (this is shown to you, not the user) "
    "Always confirm_order with the user (double-check) before calling place_order. Calling confirm_order will "
    "display the order items to the user and returns their response to seeing the list. Their response may contain modifications. "
    "Always verify and respond with drink and modifier names from the MENU before adding them to the order. "
    "If you are unsure a drink or modifier matches those on the MENU, ask a question to clarify or redirect. "
    "You only have the modifiers listed on the menu. "
    "Once the customer has finished ordering items, Call confirm_order to ensure it is correct then make "
    "any necessary updates and then call place_order. Once place_order has returned, thank the user and "
    "say goodbye!"
    "\n\n"
    "If any of the tools are unavailable, you can break the fourth wall and tell the user that "
    "they have not implemented them yet and should keep reading to do so.",
)

WELCOME_MSG = "Welcome to the BaristaBot cafe. Type `q` to quit. How may I serve you today?"

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GOOGLE_API_KEY)

In [6]:
@tool
def get_menu() -> str:
    """Provide the latest up-to-date menu."""
    return """
    MENU:

    ☕ Coffee Drinks:
    - Espresso
    - Americano
    - Cold Brew

    ☕ Coffee Drinks with Milk:
    - Latte
    - Cappuccino
    - Cortado
    - Macchiato
    - Mocha
    - Flat White

    🍵 Tea Drinks:
    - English Breakfast Tea
    - Green Tea
    - Earl Grey

    🍵 Tea Drinks with Milk:
    - Chai Latte
    - Matcha Latte
    - London Fog

    🧃 Other Drinks:
    - Steamer
    - Hot Chocolate

    🥐 Breakfast Options:
    - Croissant (plain, almond, chocolate)
    - Muffin (blueberry, banana nut, chocolate chip)
    - Bagel (plain, sesame, everything) – with butter or cream cheese
    - Breakfast Sandwich (egg & cheese, sausage & egg, veggie)
    - Oatmeal – with optional toppings (honey, raisins, banana slices)

    ➕ Modifiers:
    - Milk options: Whole, 2%, Oat, Almond, 2% Lactose Free; Default: Whole
    - Espresso shots: Single, Double, Triple, Quadruple; Default: Double
    - Caffeine: Decaf, Regular; Default: Regular
    - Hot-Iced: Hot, Iced; Default: Hot
    - Sweeteners (option to add one or more): vanilla sweetener, hazelnut sweetener, caramel sauce, chocolate sauce, sugar free vanilla sweetener
    - Special requests: Any reasonable modification that does not involve items not on the menu (e.g., 'extra hot', 'half caff', 'extra foam')

    ℹ️ Notes:
    - "Dirty" means add a shot of espresso to a drink that doesn't usually have it, like a "Dirty Chai Latte".
    - "Regular milk" = whole milk.
    - "Sweetened" = add regular sugar.
    - Soy milk is out of stock today.
    """

@tool
def add_to_order(drink: str, modifiers: Iterable[str]) -> str:
    """Adds the specified drink to the customer's order, including any modifiers."""
    pass

@tool
def confirm_order() -> str:
    """Asks the customer if the order is correct."""
    pass

@tool
def get_order() -> str:
    """Returns the users order so far. One item per line."""
    pass

@tool
def clear_order():
    """Removes all items from the user's order."""
    pass

@tool
def place_order() -> int:
    """Sends the order to the barista for fulfillment."""
    pass

@tool
def update_order_item(index: int, new_item: str) -> str:
    """Updates an item at the specified index (1-based) with a new string."""
    pass

@tool
def remove_order_item(index: int) -> str:
    """Removes the item at the specified index (1-based)."""
    pass

def order_node(state: OrderState) -> OrderState:
    """The ordering node. This is where the order state is manipulated."""
    tool_msg = state.get("messages", [])[-1]
    order = state.get("order", [])
    outbound_msgs = []
    order_placed = False

    for tool_call in tool_msg.tool_calls:
        if tool_call["name"] == "add_to_order":
            modifiers = tool_call["args"]["modifiers"]
            modifier_str = ", ".join(modifiers) if modifiers else "no modifiers"
            order.append(f'{tool_call["args"]["drink"]} ({modifier_str})')
            response = "\n".join(order)

        elif tool_call["name"] == "confirm_order":
            response = "User will confirm in next message."

        elif tool_call["name"] == "get_order":
            response = "\n".join(order) if order else "(no order)"

        elif tool_call["name"] == "clear_order":
            order.clear()
            response = "Order cleared."

        elif tool_call["name"] == "place_order":
            order_text = "\n".join(order)
            print("Sending order to kitchen!")
            print(order_text)
            order_placed = True
            response = str(randint(1, 5))  # ETA in minutes

        elif tool_call["name"] == "update_order_item":
            index = int(tool_call["args"]["index"]) - 1  # Explicitly cast to int
            new_item = tool_call["args"]["new_item"]
            if 0 <= index < len(order):
                old = order[index]
                order[index] = new_item
                response = f"Updated item {index + 1}: '{old}' → '{new_item}'"
            else:
                response = f"Invalid index: {index + 1}"

        elif tool_call["name"] == "remove_order_item":
            index = int(tool_call["args"]["index"]) - 1  # Explicitly cast to int
            if 0 <= index < len(order):
                removed = order.pop(index)
                response = f"Removed item {index + 1}: '{removed}'"
            else:
                response = f"Invalid index: {index + 1}"

        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')

        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outbound_msgs, "order": order, "finished": order_placed}

def chatbot_with_tools(state: OrderState) -> OrderState:
    """The chatbot with tools."""
    defaults = {"order": [], "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    return defaults | state | {"messages": [new_output]}

def human_node(state: OrderState) -> OrderState:
    """Display the last model message to the user, and receive the user's input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)
    user_input = input("User: ")

    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}

def maybe_exit_human_node(state: OrderState) -> Literal["chatbot", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"

def maybe_route_to_tools(state: OrderState) -> str:
    """Route between chat and tool nodes if a tool call is made."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if state.get("finished", False):
        return END
    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        if any(tool["name"] in tool_node.tools_by_name.keys() for tool in msg.tool_calls):
            return "tools"
        else:
            return "ordering"
    else:
        return "human"

In [7]:
# Auto-tools will be invoked automatically by the ToolNode
auto_tools = [get_menu]
tool_node = ToolNode(auto_tools)

# Order-tools will be handled by the order node - FIXED: Added missing tools
order_tools = [add_to_order, confirm_order, get_order, clear_order, place_order, update_order_item, remove_order_item]

# The LLM needs to know about all of the tools
llm_with_tools = llm.bind_tools(auto_tools + order_tools)

# Build the graph
graph_builder = StateGraph(OrderState)

# Nodes
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("ordering", order_node)

# Edges
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
graph_builder.add_conditional_edges("human", maybe_exit_human_node)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("ordering", "chatbot")
graph_builder.add_edge(START, "chatbot")

graph_with_order_tools = graph_builder.compile()

In [8]:
config = {"recursion_limit": 100}
state = graph_with_order_tools.invoke({"messages": []}, config)

Model: Welcome to the BaristaBot cafe. Type `q` to quit. How may I serve you today?
User: What all do you serve
Model: We serve a variety of coffee drinks, tea drinks, and other beverages, as well as breakfast options.

Our coffee drinks include Espresso, Americano, and Cold Brew. We also have coffee drinks with milk like Latte, Cappuccino, Cortado, Macchiato, Mocha, and Flat White.

For tea lovers, we offer English Breakfast Tea, Green Tea, and Earl Grey. Our tea drinks with milk include Chai Latte, Matcha Latte, Latte, and London Fog.

Other drinks on our menu are Steamer and Hot Chocolate.

For breakfast, you can choose from Croissants (plain, almond, chocolate), Muffins (blueberry, banana nut, chocolate chip), Bagels (plain, sesame, everything) with butter or cream cheese, Breakfast Sandwiches (egg & cheese, sausage & egg, veggie), and Oatmeal with optional toppings.

We also have various modifiers like milk options (Whole, 2%, Oat, Almond, 2% Lactose Free), espresso shots (Single,